#### Use this as a play ground to test your code and understand the problem better.

In [25]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from decouple import config
import time
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pickle

##### Default Scraper
Ideally you won't need to change anything here but feel free to do so if you need to

In [30]:
module = config("MODULE")
jerseySTEMUtilsLoggerURL = config("LOGGER")


class Scraper:
    # Default base URL and other constants for the scraper
    BASE_URL = config("URL")  # Default base URL, loaded from configuration
    WINDOW_SIZE = "1920x1080"  # Size of the browser window to use
    USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36"  # User agent string for the browser

    def __init__(self, base_url=None):
        """
        Initializes the Scraper object, optionally with a custom base URL.

        :param base_url: Optional; The base URL for web scraping if you want to override the default.
        """
        if base_url:
            self.BASE_URL = base_url
        self.driver = self.setup_driver()

    def logger(self, function_name, message):
        """
        Logs messages to a remote logging service.

        :param function_name: Name of the function where the log is coming from.
        :param message: The log message.
        """
        requests.post(
            jerseySTEMUtilsLoggerURL,
            json={
                "module": module,
                "function_name": function_name,
                "message": message,
            },
        )

    def setup_driver(self):
        """
        Sets up the Selenium WebDriver with custom options.

        :return: Configured Selenium WebDriver.
        """
        chrome_options = Options()
        # chrome_options.add_argument("--headless")  # Uncomment to run Chrome in headless mode.
        chrome_options.add_argument(f"--window-size={self.WINDOW_SIZE}")
        chrome_options.add_argument(f"user-agent={self.USER_AGENT}")
        chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
        chrome_options.add_argument(
            "--disable-dev-shm-usage"
        )  # Overcome limited resource problems
        driver = webdriver.Chrome(options=chrome_options)
        return driver

    def navigate(self, endpoint=""):
        """
        Navigates to a specified endpoint relative to the BASE_URL.

        :param endpoint: The relative URL to navigate to. Defaults to an empty string, which navigates to the BASE_URL.
        """
        self.driver.get(f"{self.BASE_URL}/{endpoint}")

    def wait_for_element(self, locator_type, locator, timeout=10):
        """
        Waits for a specific element to be visible on the web page.

        :param locator_type: The type of the locator (e.g., By.ID, By.XPATH).
        :param locator: The locator string.
        :param timeout: Time in seconds to wait before throwing a timeout exception. Defaults to 10 seconds.
        :return: The WebElement found.
        """
        return WebDriverWait(self.driver, timeout).until(
            EC.visibility_of_element_located((locator_type, locator))
        )

    def wait_for_clickable(self, locator_type, locator, timeout=10):
        """
        Waits for a specific element to be clickable on the web page.

        :param locator_type: The type of the locator (e.g., By.ID, By.XPATH).
        :param locator: The locator string.
        :param timeout: Time in seconds to wait before throwing a timeout exception. Defaults to 10 seconds.
        :return: The WebElement found.
        """
        return WebDriverWait(self.driver, timeout).until(
            EC.element_to_be_clickable((locator_type, locator))
        )

    def switch_to_frame(self, frame_id):
        """
        Switches the context to a specified iframe by its ID.

        :param frame_id: The ID of the iframe to switch to.
        """
        WebDriverWait(self.driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it(frame_id)
        )

    def switch_to_new_window(self):
        """
        Switches the driver context to a new browser window that was opened.
        """
        self.original_window = self.driver.current_window_handle
        WebDriverWait(self.driver, 10).until(
            lambda driver: len(driver.window_handles) > 1
        )
        new_window = [
            window
            for window in self.driver.window_handles
            if window != self.original_window
        ][-1]
        self.driver.switch_to.window(new_window)

    def switch_back_to_main_window(self):
        """
        Switches the driver context back to the original main window.
        """
        self.driver.switch_to.window(self.original_window)

    def close_driver(self):
        """
        Closes the browser and quits the driver.
        """
        self.driver.quit()

##### Cookie Handling #####

In [31]:
class CookieManager:
    def __init__(self, driver, cookies_path):
        self.driver = driver
        self.cookies_path = cookies_path

    def save_cookies(self):
        with open(self.cookies_path, 'wb') as filehandler:
            cookies_dump = self.driver.get_cookies()
            print(cookies_dump)
            pickle.dump(cookies_dump, filehandler)

    def load_cookies(self):
        with open(self.cookies_path, 'rb') as cookiesfile:
            cookies = pickle.load(cookiesfile)
            for cookie in cookies:
                if 'expiry' in cookie:
                    del cookie['expiry']
                self.driver.add_cookie(cookie)


##### Build Your Scraper #####
You will use the below as a reference and update it with your code.
to solve the problem.

In [32]:
class ICIMSAutomation(Scraper):

    def __init__(self, email, password, cookies_path):
        super().__init__()
        self.email = email
        self.password = password
        self.cookie_manager = CookieManager(self.driver, cookies_path)

    def login(self):
        self.logger("class ICIMSAutomation:login", "Logging in to ICIMS")

        try:
            self.driver.get(self.BASE_URL)
            self.wait_for_clickable(
                By.XPATH, "//a[contains(@class, 'icims-15l4veq')]"
            ).click()
            self.wait_for_element(By.NAME, "username").send_keys(self.email)
            self.driver.find_element(By.CLASS_NAME, "_button-login-id").click()
            self.wait_for_element(By.ID, "identifierId").send_keys(self.email)
            self.wait_for_clickable(By.ID, "identifierNext").click()
            self.wait_for_element(By.NAME, "password").send_keys(self.password)
            self.wait_for_clickable(By.ID, "passwordNext").click()
            print("Logged in successfully")
            self.logger("class ICIMSAutomation:login", "Logged in successfully")

        except Exception as e:
            print(f"Exception during login: {str(e)}")
            self.logger("class ICIMSAutomation:login", f"Error: {e}")

    # Everything below is business logic specific to the JerseySTEM ICIMSAutomation
    def get_created_job_id(self):
        id_element = self.wait_for_element(By.CSS_SELECTOR, "small.profile-card-id")
        id_number = id_element.text.split()[1]
        return id_number

    def post_job_external(self):
        self.logger(
            "class ICIMSAutomation:post_job_external", "Posting job to external sites"
        )

        try:
            self.wait_for_clickable(By.ID, "RQ_BROADCAST").click()
            self.switch_to_frame("display_frame_left")
            self.wait_for_clickable(By.ID, "RQ_POSTING_TO_ALL_CAREER_SITES").click()
            autocomplete_input = self.wait_for_clickable(By.ID, "posting-autocomplete")
            autocomplete_input.click()
            autocomplete_input.send_keys("External")
            self.wait_for_clickable(
                By.XPATH, "//li[contains(text(), 'External')]"
            ).click()
            autocomplete_input.send_keys(Keys.ENTER)
            self.wait_for_clickable(By.ID, "post-button").click()
            self.logger(
                "class ICIMSAutomation:post_job_external",
                "Job posted to external sites",
            )

        except Exception as e:
            print(f"Exception during posting to external sites: {str(e)}")
            self.logger("class ICIMSAutomation:post_job_external", f"Error: {e}")

    def create_new_job(self, job_template_id):
        self.logger("class ICIMSAutomation:create_new_job", "Creating new job")

        try:
            self.wait_for_clickable(By.ID, "navdropdown_0_-1_0").click()
            self.wait_for_clickable(By.ID, "navdropdown_1_0_1").click()
            self.switch_to_frame("main_body")
            custom_dropdown = self.wait_for_element(
                By.XPATH, "//a[@id='JobProfileFields.JobTemplate_icimsDropdown']"
            )
            self.driver.execute_script("arguments[0].click();", custom_dropdown)
            custom_dropdown.click()
            search_input = self.wait_for_element(
                By.XPATH, "//input[@class='dropdown-search']"
            )
            search_input.send_keys(job_template_id)
            time.sleep(2)
            self.wait_for_clickable(
                By.ID, f"result-selectable_JobProfileFields.JobTemplate_1"
            ).click()
            self.wait_for_clickable(By.ID, "nextButton").click()
            self.wait_for_clickable(By.ID, "finishButton").click()
            created_job_id = self.get_created_job_id()
            print(f"Created job ID: {created_job_id}")
            self.post_job_external()
            self.logger("class ICIMSAutomation:create_new_job", "Job created")

        except Exception as e:
            print(f"Exception during job creation: {str(e)}")
            self.logger("class ICIMSAutomation:create_new_job", f"Error: {e}")

    def remove_old_postings(self, search_templates, job_template_id):
        """Removes old job postings by unposting and moving them to a 'Closed(No fill)' folder."""

        def navigate_to_search_page():
            """Navigates to the search page."""
            self.driver.get("https://jerseystem.icims.com/platform")
            self.wait_for_clickable(By.ID, "navdropdown_0_-1_1").click()
            self.wait_for_clickable(By.ID, "navdropdown_1_1_2").click()

        def select_and_search_template(template_id):
            """Selects a search template and initiates the search."""
            self.switch_to_frame("main_body")
            custom_dropdown = self.wait_for_element(
                By.XPATH, "//a[@id='savedsearchpicker_icimsDropdown']"
            )
            self.driver.execute_script("arguments[0].click();", custom_dropdown)
            custom_dropdown.click()

            search_input = self.wait_for_element(
                By.XPATH, "//input[@class='dropdown-search']"
            )
            search_input.clear()
            search_input.send_keys(template_id)
            time.sleep(2)
            self.wait_for_clickable(
                By.ID, "result-selectable_savedsearchpicker_0"
            ).click()
            time.sleep(2)

            job_template_input = self.wait_for_element(
                By.XPATH, "//input[@placeholder='— Blank —']"
            )
            job_template_input.clear()
            job_template_input.send_keys(job_template_id)
            self.wait_for_clickable(By.ID, "searchSubmitButton").click()
            time.sleep(1)

            # Select all after ensuring the search results have loaded
            self.wait_for_clickable(
                By.XPATH,
                "//button[@class='btn btn-default dropdown-toggle' and @data-toggle='dropdown']",
            ).click()
            self.wait_for_clickable(
                By.XPATH,
                "//a[@class='dropdown-item viewAction' and @title='Select All']",
            ).click()

        def unpost_jobs():
            """Unposts the selected jobs."""
            self.wait_for_clickable(By.ID, "actionPost_anchor").click()
            self.switch_to_new_window()
            try:
                self.wait_for_clickable(By.ID, "UnpostAll3").click()
            except WebDriverException:
                # Fallback selector
                self.wait_for_clickable(By.ID, "3").click()
            self.wait_for_clickable(
                By.XPATH, "//button[.//span[text()='Save']]"
            ).click()
            self.switch_back_to_main_window()

        def move_jobs_to_closed_no_fill():
            """Moves jobs to the 'Closed(No fill)' folder."""
            self.wait_for_clickable(By.ID, "actionBulkEditFields_anchor").click()
            self.switch_to_new_window()
            self.wait_for_clickable(
                By.ID, "JobProfileFields.Folder_icimsDropdown"
            ).click()
            self.wait_for_clickable(
                By.XPATH, "//li[contains(text(), 'Closed (Not Filled)')]"
            ).click()
            self.wait_for_clickable(By.ID, "saveButton").click()
            self.switch_back_to_main_window()

        # Workflow execution
        self.logger(
            "class ICIMSAutomation:remove_old_postings",
            "Removing old job postings",
        )
        try:
            navigate_to_search_page()
            select_and_search_template(search_templates["first"])
            unpost_jobs()

            navigate_to_search_page()
            select_and_search_template(search_templates["second"])
            move_jobs_to_closed_no_fill()
            self.logger(
                "class ICIMSAutomation:remove_old_postings",
                "Old job postings removed",
            )
        except Exception as e:
            print(f"Exception during removing old postings: {str(e)}")
            self.logger(
                "class ICIMSAutomation:remove_old_postings",
                f"Error: {e}",
            )

##### Main #####
 Main binds everything and executes your automation
 
Ensure to update the env variables in the .env file(dev) or in the environment variables(production) 

In [34]:
email = config("USERNAME")
password = config("PASSWORD")
cookies_path = config("COOKIES_PATH")
job_template_id = config("JOB_TEMPLATE_ID")
job_search_templates = {
    "first": config("FIRST_SEARCH_TEMPLATE"),
    "second": config("SECOND_SEARCH_TEMPLATE"),
}

automation = ICIMSAutomation(email, password, cookies_path)

# Update login button xpath/selector for your screen
automation.login()
# The below 2 methods are business and will change based on your task
automation.create_new_job(job_template_id)
automation.remove_old_postings(job_search_templates, job_template_id)

automation.close_driver()


Logged in successfully
Created job ID: 2024-1938
